In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
Activity = pd.read_csv('../../data/Activity_202009190302.csv', low_memory=False)

In [ ]:
Activity['ActivityTime'] = pd.to_datetime( Activity['ActivityTime'] )

Activity['hour'] = Activity['ActivityTime'].dt.hour
Activity['day'] = Activity['ActivityTime'].dt.day

cat_for_merge_1 = 'Helsana+ survey second half-year 2020'
cat_for_merge_2 = 'Helsana+ survey first half year'
Activity.loc[Activity['BasicActivity'] == cat_for_merge_1, 'BasicActivity'] = 'survey'
Activity.loc[Activity['BasicActivity'] == cat_for_merge_2, 'BasicActivity'] = 'survey'

Activity['BasicActivity'].astype('category')


Activity = Activity.sort_values(by = 'ActivityTime')
Activity =Activity.reset_index(drop=True)

Activity['ActivityDetails'] = Activity['ActivityDetails'].astype('category')
# Activity['BasicActivity'] = Activity['BasicActivity'].astype('category')
# Activity['REF_PARTNER'] = Activity['REF_PARTNER'].astype('np.int')

In [ ]:
#only keeping high occurence activities
Activity = Activity.loc[Activity.groupby('BasicActivity').REF_PARTNER.transform(len) > 100]

In [ ]:
Activity['BasicActivity'] = Activity['BasicActivity'].astype('category')

In [ ]:
Activity['mock_fraud'] = np.random.randint(5, size = len(Activity))

In [ ]:
activities_list = Activity['BasicActivity'].value_counts().index

In [ ]:
# cut_off = '2020-03-12 07:32:12'
cut_off = '2020-04-12 07:32:12'
# LaterActivity = Activity.loc[Activity['ActivityTime'] > cut_off, 'ActivityTime'].hist()
LaterActivity = Activity.loc[Activity['ActivityTime'] > cut_off]
#plt.save

In [ ]:
PARTNER_ID = 430124127

Agent_time_line = LaterActivity.loc[ LaterActivity['REF_PARTNER'] == PARTNER_ID]
Agent_time_line_arr = Agent_time_line['BasicActivity'].to_numpy()

Agent_time_line_arr = Agent_time_line_arr[(Agent_time_line_arr != 'Daily session')]

trans_mat = pd.crosstab(Agent_time_line_arr[1:]\
                        , Agent_time_line_arr[:-1]\
                        ,normalize=1)


last_activity = Agent_time_line_arr[-1]

trans_mat[last_activity].idxmax()

choices = trans_mat[last_activity].index.to_numpy()

beta = 4
exps = np.exp(beta * trans_mat[last_activity])
exps /= exps.sum()
np.random.choice( choices, p=exps )

In [ ]:
trans_cols = trans_mat.columns

import matplotlib
matplotlib.rcParams.update({'font.size': 14})
fig, ax = plt.subplots()
ax.imshow( trans_mat.T )
ax.set_xticks(np.arange(len(trans_cols)))
ax.set_yticks(np.arange(len(trans_cols)))
# ... and label them with the respective list entries
# trans_cols=trans_cols[::-1]
ax.set_xticklabels(trans_cols)
ax.set_yticklabels(trans_cols)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

for i, c_i in enumerate(trans_cols):
    for j, c_j in enumerate(trans_cols):
        text = ax.text(j, i, np.round( trans_mat[c_i][c_j], 2), ha="center", va="center", color="w")
#         text = ax.text(j, i, 2, ha="center", va="center", color="w")

fig.savefig("transitio_mat.png", dpi = 200, bbox_inches='tight')

In [ ]:
time_line_arr = LaterActivity['BasicActivity'].to_numpy()


trans_mat_ult = pd.crosstab(time_line_arr[1:]\
                        , time_line_arr[:-1]\
                        ,normalize=1)


trans_mat_ult[:] = 0 

In [ ]:
PARTNER_COUNTS = np.array( Activity['REF_PARTNER'].value_counts().tolist() )
PARTNER_IDS = np.array( Activity['REF_PARTNER'].value_counts().index.tolist() )

lim = 10

PARTNER_IDS = PARTNER_IDS[PARTNER_COUNTS > lim]
PARTNER_COUNTS = PARTNER_COUNTS[ PARTNER_COUNTS > lim ]


In [ ]:
PARTNER_ID = 430124127
for PARTNER_ID in PARTNER_IDS:
    Agent_time_line = LaterActivity.loc[ LaterActivity['REF_PARTNER'] == PARTNER_ID]
    Agent_time_line_arr = Agent_time_line['BasicActivity'].to_numpy()

    Agent_time_line_arr = Agent_time_line_arr[(Agent_time_line_arr != 'Daily session')]

    trans_mat = pd.crosstab(Agent_time_line_arr[1:]\
                            , Agent_time_line_arr[:-1]\
                            ,normalize=1)
    shared = trans_mat_ult.columns & trans_mat.columns
    trans_mat_ult.loc[shared, shared] = trans_mat_ult[shared][shared] + trans_mat[shared][shared]
#     trans_mat_ult = trans_mat_ult + trans_mat
#     trans_mat_ult[trans_mat.columns & trans_mat_1.columns] + trans_mat_1[trans_mat.columns & trans_mat_1.columns] 


# last_activity = Agent_time_line_arr[-1]

# trans_mat[last_activity].idxmax()

trans_mat_ult /= trans_mat_ult.sum()

trans_mat_ult = trans_mat_ult.fillna(0)

removable_cols = trans_mat_ult.columns[np.array( [trans_mat_ult.sum() == 0 ])[0]]

trans_mat_ult = trans_mat_ult.drop(removable_cols, 1)
trans_mat_ult = trans_mat_ult.drop(removable_cols, 0)

In [ ]:
# PARTNER_COUNTS = np.array( Activity['REF_PARTNER'].value_counts().tolist() )
# PARTNER_IDS = np.array( Activity['REF_PARTNER'].value_counts().index.tolist() )

# lim = 20

# PARTNER_IDS = PARTNER_IDS[PARTNER_COUNTS > lim]
# PARTNER_COUNTS = PARTNER_COUNTS[ PARTNER_COUNTS > lim ]


In [ ]:
PARTNER_IDS = PARTNER_IDS.astype(int)


In [ ]:
recomms = pd.DataFrame(PARTNER_IDS, columns = ['REF_PARTNER'])
recomms['recom'] = 0

In [ ]:
Loyalty_activities=['Weekly Bonus', 'Discover Helsana Coach', 'survey'\
, 'Step Challenge - Helsana Coach App', 'Longstanding customers'\
, 'Link to health tracker app', 'Long-term policy', 'Medical early detection'\
, 'Helsana+ survey second half-year 2020','Helsana+ survey first half year']

trans_mat_ult.

In [ ]:
shared = trans_mat_ult.index & Loyalty_activities

In [ ]:
trans_mat_ult_alter = trans_mat_ult.drop(shared, 0)

In [ ]:
for PARTNER_ID in PARTNER_IDS:
    Agent_time_line = LaterActivity.loc[ LaterActivity['REF_PARTNER'] == PARTNER_ID]
    Agent_time_line_arr = Agent_time_line['BasicActivity'].to_numpy()
    
    Agent_time_line_arr = Agent_time_line_arr[(Agent_time_line_arr != 'Daily session')]

    if(len(Agent_time_line_arr)):
        last_activity = Agent_time_line_arr[-1]
    else:
        last_activity = activities_list[0]
        
    choices = trans_mat_ult_alter[last_activity].index.to_numpy()

    beta = 2
    exps = np.exp(beta * trans_mat_ult_alter[last_activity])
    exps /= exps.sum()
    recom = np.random.choice( choices, p=exps )
    recomms.loc[recomms['REF_PARTNER'] == PARTNER_ID, 'recom'] = recom

In [ ]:
choices

In [ ]:
recomms['recom'].value_counts()

In [ ]:
recomms.to_csv('Recomms.csv', index = False)

In [ ]:
trans_cols = trans_mat_ult_alter.columns;
trans_rows = trans_mat_ult_alter.index

import matplotlib
matplotlib.rcParams.update({'font.size': 11})
fig, ax = plt.subplots()
ax.imshow( trans_mat_ult_alter.T );
ax.set_xticks(np.arange(len(trans_rows)))
ax.set_yticks(np.arange(len(trans_cols)));
# # ... and label them with the respective list entries
# # trans_cols=trans_cols[::-1]
ax.set_xticklabels(trans_rows)
ax.set_yticklabels(trans_cols);
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor");

# # for i, c_i in enumerate(trans_cols):
# #     for j, c_j in enumerate(trans_cols):
# #         text = ax.text(j, i, np.round( trans_mat_ult[c_i][c_j], 1), ha="center", va="center", color="w")


# fig.savefig("transitio_mat_ult-2.png", dpi = 200, bbox_inches='tight')

In [ ]:
trans_cols = trans_mat_ult.columns

import matplotlib
matplotlib.rcParams.update({'font.size': 11})
fig, ax = plt.subplots()
ax.imshow( trans_mat_ult.T )
ax.set_xticks(np.arange(len(trans_cols)))
ax.set_yticks(np.arange(len(trans_cols)))
# ... and label them with the respective list entries
# trans_cols=trans_cols[::-1]
ax.set_xticklabels(trans_cols)
ax.set_yticklabels(trans_cols)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# for i, c_i in enumerate(trans_cols):
#     for j, c_j in enumerate(trans_cols):
#         text = ax.text(j, i, np.round( trans_mat_ult[c_i][c_j], 1), ha="center", va="center", color="w")


fig.savefig("transitio_mat_ult-2.png", dpi = 200, bbox_inches='tight')

In [ ]:
activity_list = np.array(Activity['BasicActivity'].value_counts().index)

In [ ]:
# activity_list = ['Daily session', 'Nutrition programme', 'Trails App Run', 'Personal exercise', 'Weekly bonus']

fig , ax = plt.subplots()
for specific_activity in activity_list:
    Agent_specific_activity = Agent_time_line.loc[Agent_time_line['BasicActivity']== specific_activity]
    if len(Agent_specific_activity):
        ax.scatter(Agent_specific_activity['ActivityTime'], Agent_specific_activity['hour']\
                 , label = specific_activity, alpha = 0.8, edgecolor = 'k', s=45)

fraudaluent = Agent_time_line.loc[Agent_time_line['mock_fraud'] == 1]
ax.scatter( fraudaluent['ActivityTime'], fraudaluent['hour'], facecolors = 'none'\
           , linestyle = '--', edgecolor = 'r', s = 100, label = 'fraud', linewidth = 2 )
    
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

time_range = pd.date_range(Agent_time_line['ActivityTime'].to_numpy()[0]\
                                    , Agent_time_line['ActivityTime'].to_numpy()[-1], periods=8)
ax.plot(time_range, np.ones(len(time_range)),'-o')


fig.savefig("specificAgentt.png", dpi = 300, bbox_inches='tight')

In [ ]:
# begin = Agent_time_line['ActivityTime'].to_numpy()[0]
# end =  Agent_time_line['ActivityTime'].to_numpy()[-1]
# periods = float( (end - begin) / 1000000000/`3600/24 )
# time_range = pd.date_range(begin, end, periods=periods)
# y = np.sin(np.arange(len(time_range)))
# plt.plot(time_range, y,'--')
